In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers datasets accelerate -q


In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn import BCEWithLogitsLoss
from tqdm.auto import tqdm


In [ ]:
df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")
df = df[['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']].dropna()
df = df.astype(int, errors='ignore')  # convert labels to int

dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tok(batch):
    return tokenizer(batch["comment_text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tok, batched=True)

tokenized.set_format(
    type="torch",
    columns=["input_ids","attention_mask",
             "toxic","severe_toxic","obscene",
             "threat","insult","identity_hate"]
)


In [ ]:
train_loader = DataLoader(tokenized['train'], batch_size=16, shuffle=True)
val_loader = DataLoader(tokenized['test'], batch_size=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    problem_type="multi_label_classification"
).to(device)

criterion = BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


In [ ]:
model.train()
epochs = 1

for epoch in range(epochs):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for batch in pbar:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        labels = torch.stack([
            batch["toxic"],
            batch["severe_toxic"],
            batch["obscene"],
            batch["threat"],
            batch["insult"],
            batch["identity_hate"]
        ], dim=1).float().to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss": loss.item()})


In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        labels = torch.stack([
            batch["toxic"],
            batch["severe_toxic"],
            batch["obscene"],
            batch["threat"],
            batch["insult"],
            batch["identity_hate"]
        ], dim=1).to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = (torch.sigmoid(outputs.logits) > 0.5).int()

        correct += (preds == labels).all(dim=1).sum().item()
        total += labels.size(0)

print("Validation Accuracy:", correct/total)


In [ ]:
label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

def classify_comment(text, toxic_threshold=0.5, category_threshold=0.5):
    model.eval()
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    probs = torch.sigmoid(outputs.logits)[0].cpu().numpy()
    
    toxic_prob = float(probs[0])
    
    print("\n📝 Comment:", text)
    print(f"🔥 Toxic Probability: {toxic_prob:.4f}")
    
    if toxic_prob >= toxic_threshold:
        print("Prediction: ⚠️ Toxic")
        
        categories = [label_names[i] for i in range(1, 6) if probs[i] >= category_threshold]
        
        if categories:
            print("🧩 Toxicity Type(s):", ", ".join(categories))
        else:
            print("🧩 Toxicity Type: General Toxicity")
    else:
        print("Prediction: 🙂 Non-Toxic")


In [ ]:
save_path = "/kaggle/working/toxic_multilabel_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved to:", save_path)


In [ ]:
!pwd


In [ ]:
!ls -lh /kaggle/working


In [ ]:
!ls -lh /kaggle/working/toxic_multilabel_model


In [ ]:
!zip -r toxic_multilabel_model.zip /kaggle/working/toxic_multilabel_model
